<a href="https://colab.research.google.com/github/RobBurnap/Bioinformatics-MICR4203-MICR5203/blob/main/notebooks/Species_Tree_Diversity_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# BIOINFO4/5203 —
species diveristy:

##A. Mount Google Drive, Import Coding Libraries Necessary for Running Subsequent Code

In [2]:

# Install FIRST, then import
%pip install -q biopython       # Install the Biopython package quietly (-q suppresses most output) so we can work with biological sequence files

from google.colab import drive  # Import the module that lets Colab interact with Google Drive
drive.mount('/content/drive')   # Mount your Google Drive so it appears in Colab's file system under /content/drive

import os, pandas as pd          # Import 'os' for file/directory operations, and pandas for working with data tables
from Bio import SeqIO            # Import SeqIO from Biopython for reading/writing biological sequence files (FASTA, GenBank, etc.)
import matplotlib.pyplot as plt  # Import Matplotlib's plotting library to create figures and graphs

print("✅ Dependencies installed & Drive mounted.")


Mounted at /content/drive
✅ Dependencies installed & Drive mounted.



## B. Course folders: Define the course folders for places to load data to be processed and output to be saved

Edit only `LECTURE_CODE` and `TOPIC` if needed. All inputs will live in `Data/LECTURE_TOPIC` and outputs in `Outputs/LECTURE_TOPIC`.


In [3]:

# --- Course folder config (customize LECTURE_CODE/TOPIC only) ---
COURSE_DIR   = "/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25"
LECTURE_CODE = "L0-species"            # change per week (e.g., L02, L03, ...)
TOPIC        = "diversity"    # short slug for the exercise

# Derived paths (do not change)
DATA_DIR   = f"{COURSE_DIR}/Data/{LECTURE_CODE}_{TOPIC}"
OUTPUT_DIR = f"{COURSE_DIR}/Outputs/{LECTURE_CODE}_{TOPIC}"

# Create folder structure if missing
for p in [f"{COURSE_DIR}/Data", f"{COURSE_DIR}/Outputs", f"{COURSE_DIR}/Notebooks", DATA_DIR, OUTPUT_DIR]:
    os.makedirs(p, exist_ok=True)

print("📁 COURSE_DIR :", COURSE_DIR)
print("📁 DATA_DIR   :", DATA_DIR)
print("📁 OUTPUT_DIR :", OUTPUT_DIR)


📁 COURSE_DIR : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25
📁 DATA_DIR   : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L0-species_diversity
📁 OUTPUT_DIR : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity


##C.



 multi-FASTA of top hits per TaxID (one or more sequences per taxon). The cell below:
	•	uses your existing folders (Data/L0-species_diversity for input; writes to the same folder unless OUTPUT_DIR is already set),
	•	reads query_proteins.fasta and taxids.txt,
	•	for each TaxID, runs a separate BLAST restricted to that taxon (txid####[ORGN]), so we can attribute hits unambiguously,
	•	grabs the top N accessions from each BLAST,
	•	fetches their protein FASTA sequences,
	•	writes:
	•	per_taxid_top_hits.fasta (all sequences, grouped by taxid in headers),
	•	per_taxid_hits.tsv (who came from which taxid, evalue, %id, etc.),
	•	optional one FASTA per taxid (toggle with WRITE_SPLIT_FASTA).

In [4]:
# --- BLAST (per TaxID) -> collect top protein hits -> write multi-FASTA + TSV (fixed mode detection) ---
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from pathlib import Path
import io, csv, re, time, sys

# ==== required ====
Entrez.email = "you@university.edu"   # <-- set your email
# Entrez.api_key = "YOUR_NCBI_API_KEY"  # optional, faster/higher limits

# ==== knobs you can tweak ====
TOP_HITS_PER_TAXID = 2          # how many sequences to keep per taxid
EVALUE              = 1e-5      # keep stringent
HITLIST_SIZE        = max(50, TOP_HITS_PER_TAXID*10)
WRITE_SPLIT_FASTA   = False
SLEEP_BETWEEN_CALLS = 0.3

# Optional hard override: 'auto' | 'blastp' | 'blastx'
FORCE_MODE = 'auto'   # set to 'blastp' if your query file is protein but misdetected

# ==== paths (use your course vars if present) ====
if 'DATA_DIR' in globals():
    DATA_DIR = Path(DATA_DIR)
else:
    DATA_DIR = Path("/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L0-species_diversity")

if 'OUTPUT_DIR' in globals():
    OUTPUT_DIR = Path(OUTPUT_DIR)
else:
    OUTPUT_DIR = DATA_DIR

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Files (per your folder)
PROT_FASTA = DATA_DIR / "query_proteins.fasta"
NUC_FASTA  = DATA_DIR / "query.fasta"
TAXIDS_TXT = DATA_DIR / "taxids.txt"

# ==== helpers: accession parsing & normalization ====
def extract_accession(hit_id, hit_def, accession_attr):
    if accession_attr:
        return accession_attr.strip()
    for field in (hit_id, hit_def):
        m = re.search(r"([A-Z]{1,5}_?\d+(?:\.\d+)?)", field or "")
        if m:
            return m.group(1)
    return (hit_id or "unknown").strip()

def _norm_keys(acc_token: str):
    acc_token = acc_token.strip()
    base = acc_token.split(".", 1)[0]
    if "." in acc_token:
        return {acc_token, base}
    else:
        return {acc_token, f"{base}.1"}

def fetch_protein_fasta(accessions):
    out = {}
    batch = list({a for a in accessions if a and a != "unknown"})
    while batch:
        chunk = batch[:50]; batch = batch[50:]
        try:
            h = Entrez.efetch(db="protein", id=",".join(chunk), rettype="fasta", retmode="text")
            txt = h.read(); h.close()
            parts = [t for t in txt.strip().split(">") if t]
            for rec_txt in parts:
                header, *seq_lines = rec_txt.splitlines()
                token = header.split()[0]  # e.g., XOB97566.1 or WP_12345.1
                fasta_txt = ">" + header + "\n" + "\n".join(seq_lines) + "\n"
                for k in _norm_keys(token):
                    out[k] = fasta_txt
        except Exception as e:
            sys.stderr.write(f"[warn] efetch failed for {chunk}: {e}\n")
        time.sleep(SLEEP_BETWEEN_CALLS)
    return out

# ==== robust query loader/detector ====
DNA_ALPHABET = set("ACGTNUWSMKRYBDHV-")  # IUPAC DNA (upper)
def detect_is_protein(seq_upper: str) -> bool:
    letters = [c for c in seq_upper if c.isalpha() or c == '*']
    # If there's ANY character outside DNA alphabet (e.g., E, F, L, Q, Z, J, O), it's protein.
    return any((c not in DNA_ALPHABET) for c in letters)

def load_query():
    # Prefer explicit protein file if present
    if PROT_FASTA.exists():
        rec = next(SeqIO.parse(str(PROT_FASTA), "fasta"))
        seq = str(rec.seq).upper()
        is_prot = detect_is_protein(seq)
        decided = 'blastp' if is_prot else 'blastx'
        if FORCE_MODE in ('blastp','blastx'):
            decided = FORCE_MODE
        return decided, seq.replace("*",""), f"{PROT_FASTA.name}:{rec.id}"

    # Fallback to query.fasta
    if NUC_FASTA.exists():
        rec = next(SeqIO.parse(str(NUC_FASTA), "fasta"))
        seq = str(rec.seq).upper()
        is_prot = detect_is_protein(seq)
        decided = 'blastp' if is_prot else 'blastx'
        if FORCE_MODE in ('blastp','blastx'):
            decided = FORCE_MODE
        # Warn if filename suggests nucleotide but content looks protein
        if decided == 'blastp' and NUC_FASTA.name == "query.fasta":
            print("⚠️  Detected protein sequence in 'query.fasta'; using BLASTP. "
                  "If this is actually nucleotide, set FORCE_MODE='blastx'.")
        return decided, seq.replace("*",""), f"{NUC_FASTA.name}:{rec.id}"

    raise FileNotFoundError(f"Could not find {PROT_FASTA} or {NUC_FASTA}")

mode, query_seq, query_label = load_query()
print(f"📄 Query source: {query_label}")
print(f"🧪 Mode chosen: {mode.upper()} vs nr  | length={len(query_seq)}")

# ==== load TaxIDs ====
if not TAXIDS_TXT.exists():
    raise FileNotFoundError(f"taxids.txt not found at {TAXIDS_TXT}")
taxids = [t.strip() for t in TAXIDS_TXT.read_text().splitlines() if t.strip().isdigit()]
if not taxids:
    raise ValueError("taxids.txt is empty or contains no numeric TaxIDs.")
print(f"🧬 Loaded {len(taxids)} TaxIDs")

# ==== BLAST per-taxid ====
def run_single_blast(seq, taxid, program="blastp", evalue=EVALUE, hitlist=HITLIST_SIZE):
    q = f"txid{taxid}[ORGN]"
    preview = q if len(q) < 160 else (q[:157] + " …")
    print(f"⏳ {program.upper()} vs nr | taxid={taxid} | E={evalue} | hits={hitlist}\n   ENTREZ_QUERY: {preview}")
    h = NCBIWWW.qblast(program=program, database="nr", sequence=seq,
                       expect=evalue, entrez_query=q,
                       hitlist_size=hitlist, descriptions=hitlist, alignments=hitlist)
    xml = h.read(); h.close()
    rec = NCBIXML.read(io.StringIO(xml))
    return rec, xml

# ==== main loop -> XML + top hits -> fetch FASTA ====
all_rows = []
per_taxid_fastas = {}   # taxid -> list of FASTA strings
xml_paths = []

for i, tid in enumerate(taxids, 1):
    try:
        record, xml = run_single_blast(query_seq, tid, program=("blastp" if mode=="blastp" else "blastx"))
        xml_file = OUTPUT_DIR / f"{mode}_nr_taxid{tid}.xml"
        xml_file.write_text(xml); xml_paths.append(xml_file)

        if not record.alignments:
            print(f"— No hits for taxid {tid}")
            continue

        # Collect best HSP per alignment; rank by (evalue, -pct_id)
        hsps = []
        for aln in record.alignments:
            best = sorted(aln.hsps, key=lambda h: (h.expect, -h.identities))[0]
            acc = extract_accession(aln.hit_id, aln.hit_def, getattr(aln, "accession", None))
            pct_id = 100.0 * best.identities / best.align_length if best.align_length else 0.0
            hsps.append((aln, best, acc, pct_id))
        hsps.sort(key=lambda t: (t[1].expect, -t[3]))
        keep = hsps[:TOP_HITS_PER_TAXID]

        # Fetch FASTAs for these accessions (store under both base/version keys)
        accs = [acc for _,_,acc,_ in keep]
        acc_to_fa = fetch_protein_fasta(accs)

        # Store chosen sequences
        per_taxid_fastas.setdefault(tid, [])
        kept_now = 0
        for aln, best, acc, pct in keep:
            fa = None
            for k in _norm_keys(acc):
                fa = acc_to_fa.get(k)
                if fa: break
            if not fa:
                sys.stderr.write(f"[miss] No FASTA for {acc} (taxid {tid})\n")
                continue

            # Prepend taxid info to header
            lines = fa.strip().splitlines()
            header = lines[0][1:]  # drop '>'
            seq = "\n".join(lines[1:])
            new_header = f">taxid:{tid}|acc:{acc}|e:{best.expect:.2e}|pid:{pct:.2f}|len:{best.align_length} {header}"
            per_taxid_fastas[tid].append(new_header + "\n" + seq + "\n"); kept_now += 1

            all_rows.append([
                tid, acc, aln.title, aln.length, best.expect,
                best.identities, best.align_length, round(pct,2),
                min(best.query_start,best.query_end), max(best.query_start,best.query_end),
                min(best.sbjct_start,best.sbjct_end), max(best.sbjct_start,best.sbjct_end)
            ])
        print(f"✅ taxid {tid}: kept {kept_now} sequences")
    except Exception as e:
        print(f"⚠️ taxid {tid} failed: {e}")
    time.sleep(SLEEP_BETWEEN_CALLS)

# ==== write combined multi-FASTA + per-taxid FASTAs ====
multi_fa = OUTPUT_DIR / "per_taxid_top_hits.fasta"
with open(multi_fa, "w") as fh:
    for tid in taxids:
        for fa in per_taxid_fastas.get(tid, []):
            fh.write(fa)
print(f"💾 Multi-FASTA: {multi_fa}")

if WRITE_SPLIT_FASTA:
    for tid, fas in per_taxid_fastas.items():
        p = OUTPUT_DIR / f"taxid_{tid}_top_hits.fasta"
        with open(p, "w") as fh:
            for fa in fas: fh.write(fa)
    print("💾 Also wrote per-taxid FASTAs")

# ==== write table ====
tsv = OUTPUT_DIR / "per_taxid_hits.tsv"
with open(tsv, "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["taxid","accession","title","subject_length","evalue","identities","align_len","pct_identity","q_start","q_end","s_start","s_end"])
    w.writerows(all_rows)
print(f"📑 Table: {tsv}")

print(f"🗂 XML files saved: {len(xml_paths)}")

📄 Query source: query_proteins.fasta:cyt-c6
🧪 Mode chosen: BLASTP vs nr  | length=111
🧬 Loaded 123 TaxIDs
⏳ BLASTP vs nr | taxid=271 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid271[ORGN]
✅ taxid 271: kept 2 sequences
⏳ BLASTP vs nr | taxid=277 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid277[ORGN]
✅ taxid 277: kept 2 sequences
⏳ BLASTP vs nr | taxid=562 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid562[ORGN]
— No hits for taxid 562
⏳ BLASTP vs nr | taxid=1148 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1148[ORGN]


[miss] No FASTA for 9KRR_A (taxid 1148)


✅ taxid 1148: kept 1 sequences
⏳ BLASTP vs nr | taxid=1179 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1179[ORGN]
✅ taxid 1179: kept 2 sequences
⏳ BLASTP vs nr | taxid=1423 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1423[ORGN]
✅ taxid 1423: kept 1 sequences
⏳ BLASTP vs nr | taxid=6204 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid6204[ORGN]
— No hits for taxid 6204
⏳ BLASTP vs nr | taxid=6205 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid6205[ORGN]
— No hits for taxid 6205
⏳ BLASTP vs nr | taxid=6207 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid6207[ORGN]
— No hits for taxid 6207
⏳ BLASTP vs nr | taxid=6210 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid6210[ORGN]
— No hits for taxid 6210
⏳ BLASTP vs nr | taxid=6211 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid6211[ORGN]
— No hits for taxid 6211
⏳ BLASTP vs nr | taxid=6645 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid6645[ORGN]
— No hits for taxid 6645
⏳ BLASTP vs nr | taxid=7010 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid7010[ORGN]
— No hits for taxid 7010
⏳ BLASTP vs 

/usr/local/lib/python3.12/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request AD78U83Z015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


— No hits for taxid 9785
⏳ BLASTP vs nr | taxid=9823 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid9823[ORGN]
— No hits for taxid 9823
⏳ BLASTP vs nr | taxid=9913 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid9913[ORGN]
— No hits for taxid 9913
⏳ BLASTP vs nr | taxid=9940 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid9940[ORGN]
— No hits for taxid 9940
⏳ BLASTP vs nr | taxid=10090 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid10090[ORGN]
— No hits for taxid 10090
⏳ BLASTP vs nr | taxid=10116 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid10116[ORGN]
— No hits for taxid 10116
⏳ BLASTP vs nr | taxid=32046 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid32046[ORGN]
✅ taxid 32046: kept 2 sequences
⏳ BLASTP vs nr | taxid=34597 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid34597[ORGN]
— No hits for taxid 34597
⏳ BLASTP vs nr | taxid=37636 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid37636[ORGN]
✅ taxid 37636: kept 2 sequences
⏳ BLASTP vs nr | taxid=39669 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid39669[ORGN]
— No hits for taxid 39669


/usr/local/lib/python3.12/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request ADJZ6CR3015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


— No hits for taxid 2812896
⏳ BLASTP vs nr | taxid=2814275 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2814275[ORGN]
✅ taxid 2814275: kept 2 sequences
⏳ BLASTP vs nr | taxid=2823368 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2823368[ORGN]
— No hits for taxid 2823368
⏳ BLASTP vs nr | taxid=2833567 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2833567[ORGN]
— No hits for taxid 2833567
⏳ BLASTP vs nr | taxid=2847862 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2847862[ORGN]
✅ taxid 2847862: kept 2 sequences
⏳ BLASTP vs nr | taxid=2857077 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2857077[ORGN]


/usr/local/lib/python3.12/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request ADMSRCN7015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


✅ taxid 2857077: kept 1 sequences
⏳ BLASTP vs nr | taxid=2886104 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2886104[ORGN]
✅ taxid 2886104: kept 2 sequences
⏳ BLASTP vs nr | taxid=2886352 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2886352[ORGN]
✅ taxid 2886352: kept 2 sequences
⏳ BLASTP vs nr | taxid=2908145 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2908145[ORGN]
✅ taxid 2908145: kept 1 sequences
⏳ BLASTP vs nr | taxid=2908146 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2908146[ORGN]
✅ taxid 2908146: kept 1 sequences
⏳ BLASTP vs nr | taxid=2908147 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2908147[ORGN]
✅ taxid 2908147: kept 2 sequences
⏳ BLASTP vs nr | taxid=2922243 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2922243[ORGN]
— No hits for taxid 2922243
⏳ BLASTP vs nr | taxid=2929567 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2929567[ORGN]
✅ taxid 2929567: kept 2 sequences
⏳ BLASTP vs nr | taxid=2964529 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2964529[ORGN]
✅ taxid 2964529: kept 2 sequences
⏳ BLASTP vs 

Extracts fasta sequences from XML files

In [ ]:
from Bio.Blast import NCBIXML
from Bio import Entrez
from pathlib import Path
import re, time

# REQUIRED
Entrez.email = "your_email@university.edu"   # <-- change me
# Entrez.api_key = "YOUR_NCBI_API_KEY"       # optional but helpful

# Paths (match your screenshots)
# DATA_DIR  = Path("/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L0-species_diversity")
# OUT_FASTA = DATA_DIR / "blast_hits.fasta"

# Use the OUTPUT_DIR defined in the previous cell
if 'OUTPUT_DIR' in globals():
    OUTPUT_DIR = Path(OUTPUT_DIR)
else:
    # Fallback if the variable is not set
    OUTPUT_DIR = Path("/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity")

OUT_FASTA = OUTPUT_DIR / "blast_hits.fasta"


# Find XML files (your files look like 'blastp_nr_taxid*.xml')
# XMLS = sorted(DATA_DIR.glob("blastp_nr_taxid*.xml"))
# if not XMLS:
#     XMLS = sorted(DATA_DIR.glob("*.xml"))
# assert XMLS, f"No XML files found in {DATA_DIR}"

# Look for XML files in the OUTPUT_DIR
XMLS = sorted(OUTPUT_DIR.glob("blastp_nr_taxid*.xml"))
if not XMLS:
    XMLS = sorted(OUTPUT_DIR.glob("*.xml")) # Fallback to any xml if specific not found

assert XMLS, f"No XML files found in {OUTPUT_DIR}"


MAX_HITS_PER_XML = 5     # top N alignments per XML
BATCH = 50               # efetch batch size
SLEEP = 0.25             # courtesy to NCBI

def parse_top_accessions(xml_path, top_n=5):
    """Return list of accession strings from top alignments in one BLAST XML."""
    with open(xml_path) as h:
        rec = NCBIXML.read(h)
    accs = []
    for aln in rec.alignments[:top_n]:
        # Prefer parser's accession if available; else regex from id/def/title
        acc = getattr(aln, "accession", None)
        if not acc:
            for field in (aln.hit_id, aln.hit_def, aln.title):
                m = re.search(r"([A-Z]{1,5}_?\d+(?:\.\d+)?)", field or "")
                if m:
                    acc = m.group(1); break
        if acc:
            accs.append(acc)
    return accs

def normalize_pair(acc):
    """Return (base, versioned) forms so XOB97566 and XOB97566.1 both match."""
    base = acc.split(".")[0]
    ver  = acc if "." in acc else None
    return base, ver

def fetch_fasta_batched(accessions):
    """Fetch FASTA for a list of accessions; store under both base & version keys."""
    out = {}
    accs = [a for a in set(accessions) if a]
    for i in range(0, len(accs), BATCH):
        chunk = accs[i:i+BATCH]
        try:
            handle = Entrez.efetch(db="protein", id=",".join(chunk), rettype="fasta", retmode="text")
            text = handle.read(); handle.close()
            for block in [b for b in text.strip().split(">") if b]:
                header, *seq_lines = block.splitlines()
                header = header.strip()
                seq = "\n".join(seq_lines).strip()
                token = header.split()[0]        # e.g., XOB97566.1
                base = token.split(".")[0]
                fasta = f">{header}\n{seq}\n"
                out[token] = fasta
                out[base]  = fasta
        except Exception as e:
            print(f"[warn] efetch failed for {chunk}: {e}")
        time.sleep(SLEEP)
    return out

# --- Collect top accessions from all XMLs
wanted = []
for x in XMLS:
    accs = parse_top_accessions(x, MAX_HITS_PER_XML)
    if accs:
        wanted.extend(accs)
    else:
        print(f"— {x.name}: no alignments or no accessions parsed")

# Also include versionless forms to improve retrieval
expanded = []
for a in set(wanted):
    base, ver = normalize_pair(a)
    expanded.append(base)
    if ver: expanded.append(ver)

# --- Fetch FASTA
acc_to_fasta = fetch_fasta_batched(expanded)

# --- Write combined FASTA (keep original headers; add source xml in a comment line)
written = 0
with open(OUT_FASTA, "w") as out:
    for x in XMLS:
        accs = parse_top_accessions(x, MAX_HITS_PER_XML)
        for acc in accs:
            # try versioned, then base
            fa = acc_to_fasta.get(acc) or acc_to_fasta.get(acc.split(".")[0])
            if not fa:
                print(f"[miss] No FASTA for {acc} (from {x.name})")
                continue
            # insert a comment line indicating source XML
            header, *seq_lines = fa.strip().splitlines()
            out.write(header + f"  ; source={x.name}\n")
            out.write("\n".join(seq_lines) + "\n")
            written += 1

print(f"💾 Wrote {written} sequences to {OUT_FASTA}")

💾 Wrote 12 sequences to /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity/blast_hits.fasta
